# Register model and deploy locally

See: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local.ipynb

- Register model
- Deploy the image as a web service in a local Docker container.
- Quickly test changes to your entry script by reloading the local service.
- Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Assumptions

- Data is already preprocessed in `data/`

# Setup Workspace and Prereqs

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.50.0


In [2]:
from azureml.core.workspace import Workspace

# the config file is one directory up
# NOTE: this is an interactive authorization!
ws = Workspace.from_config('..')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Nanostics_ML_Workspace
SummerProjects2023
canadacentral
1b1ae7cf-df24-428b-8bb9-e4dd07869ac9


# Train Model

the code is ripped from `../train.ipynb`, just without the MLFlow stuff

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd 

def split_and_scale(df):
    y = df["diagnosis_01"]
    X = df.drop(["diagnosis_01"],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, X.columns

In [4]:
from sklearn.svm import SVC

def train_svc(df):
    X_train, X_test, y_train, y_test, train_cols = split_and_scale(df)
    svc = SVC()
    svc.fit(X_train, y_train)
    return svc, X_test, y_test, train_cols

In [5]:
import pandas as pd 


data = '../data/cleaned-wisconsin-lof.parquet'
df = pd.read_parquet(data)

svc, X_test, y_test, train_cols = train_svc(df)

In [6]:
# df_test = pd.DataFrame(X_test, columns=df.columns.drop('diagnosis_01'))

# d = df_test.iloc[0].to_dict()
# # # pd.DataFrame(d, index=[0])
# # svc.predict(pd.DataFrame(d, index=[0]))
X_test[0]

array([-1.24166953,  0.01754875, -0.39269772, -0.97808734, -0.775852  ,
       -0.59852503, -0.29799549, -0.82297132, -0.92586761, -0.75175096,
       -0.61798156, -0.32492961, -0.77349834, -1.07402915, -0.65012451,
       -0.82191846, -0.96269706, -1.30110812, -0.55051175, -0.58306966,
       -0.11905151, -0.68443597, -1.07128996, -0.78468918, -0.86829237,
       -0.85240946])

In [7]:
import joblib

joblib.dump(svc, 'model.pkl')

['model.pkl']

## Register Model

In [8]:
from azureml.core.model import Model

model = Model.register(model_path="model.pkl",
                       model_name="wisconsin-BCa-model",
                       tags={'area': "cancer", 'type': "svc"},
                       description="SVC model to predict Breast Cancer",
                       workspace=ws)

Registering model wisconsin-BCa-model


## Create Environment + Inference Config

straight copy-paste

In [9]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [10]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

## Deploy Model as a Docker Service

This is the good stuff!!!!

In [11]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

/tmp/ipykernel_1399/1718659047.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)


Generating Docker build context.
2023/05/16 16:01:21 Downloading source code...
2023/05/16 16:01:22 Finished downloading source code
2023/05/16 16:01:23 Creating Docker network: acb_default_network, driver: 'bridge'
2023/05/16 16:01:23 Successfully set up Docker network: acb_default_network
2023/05/16 16:01:23 Setting up Docker configuration...
2023/05/16 16:01:23 Successfully set up Docker configuration
2023/05/16 16:01:23 Logging in to registry: nanosticsmlregistry.azurecr.io
2023/05/16 16:01:24 Successfully logged into nanosticsmlregistry.azurecr.io
2023/05/16 16:01:24 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/05/16 16:01:24 Scanning for dependencies...
2023/05/16 16:01:25 Successfully scanned dependencies
2023/05/16 16:01:25 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB
Step 1/21 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230324.v1@sha256:baf6420f6eac8aa7

In [12]:
print('Local service port: {}'.format(local_service.port))


Local service port: 6789


## Test Web Service


In [ ]:
print(local_service.get_logs())


In [17]:
import json

sample_input = json.dumps({
    'data': X_test[0:2].tolist()
})

local_service.run(sample_input)

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff6dabaa30>: Failed to establish a new connection: [Errno 111] Connection refused'))